In [1]:
%%writefile testingscheme.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

#File Reading#

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0



Writing testingscheme.py


# **Write YAML file**

In [24]:
%%writefile California_realestate.yaml
file_type: csv
dataset_name: California_realestate
file_name: /gdrive/My Drive/RealEstate_California
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - id
    - stateId
    - countyId
    - cityId
    - country
    - datePostString
    - is_bankOwned
    - is_forAuction

Overwriting California_realestate.yaml


In [25]:
# Read config file
import testingscheme as util
config_data = util.read_config_file("California_realestate.yaml")

In [26]:
config_data['inbound_delimiter']

','

In [27]:
#inspecting data of config file
config_data

{'columns': ['id',
  'stateId',
  'countyId',
  'cityId',
  'country',
  'datePostString',
  'is_bankOwned',
  'is_forAuction'],
 'dataset_name': 'California_realestate',
 'file_name': '/gdrive/My Drive/RealEstate_California',
 'file_type': 'csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'table_name': 'edsurv'}

In [28]:
from google.colab import drive 

In [29]:
drive.mount('/gdrive', force_remount=True) 

Mounted at /gdrive


In [30]:
# Reading process of the file
import pandas as pd
RealEstate_California = pd.read_csv('/gdrive/My Drive/RealEstate_California.csv', sep=',')
RealEstate_California.head(3) 

,Unnamed: 0,id,stateId,countyId,cityId,country,datePostedString,is_bankOwned,is_forAuction,event,...,parking,garageSpaces,hasGarage,levels,pool,spa,isNewConstruction,hasPetsAllowed,homeType,county
0,0,95717-2087851113,9,77,24895,USA,2021-01-13,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,LOT,Placer County
1,1,94564-18496265,9,189,36958,USA,2021-07-12,0,0,Listed for sale,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
2,2,94564-18484475,9,190,36958,USA,2021-07-08,0,0,Listed for sale,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County


In [38]:
# read the file using config file
file_type = config_data['file_type']
source_file = config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,id,stateId,countyId,cityId,country,datePostedString,is_bankOwned,is_forAuction,event,...,parking,garageSpaces,hasGarage,levels,pool,spa,isNewConstruction,hasPetsAllowed,homeType,county
0,0,95717-2087851113,9,77,24895,USA,2021-01-13,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,LOT,Placer County
1,1,94564-18496265,9,189,36958,USA,2021-07-12,0,0,Listed for sale,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
2,2,94564-18484475,9,190,36958,USA,2021-07-08,0,0,Listed for sale,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
3,3,94564-18494835,9,191,36958,USA,2021-07-07,0,0,Listed for sale,...,1,1.0,1,Two Story,0,1,0,0,SINGLE_FAMILY,Contra Costa County
4,4,94564-2069722747,9,192,36958,USA,2021-07-07,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,LOT,Contra Costa County


In [33]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['latitude', 'garagespaces', 'livingarea', 'description', 'livingareavalue', 'county', 'event', 'unnamed_0', 'streetaddress', 'datepostedstring', 'hometype', 'isnewconstruction', 'price', 'longitude', 'city', 'time', 'state', 'lotareaunits', 'hasbadgeocode', 'bathrooms', 'levels', 'parking', 'bedrooms', 'currency', 'zipcode', 'yearbuilt', 'pricepersquarefoot', 'spa', 'buildingarea', 'haspetsallowed', 'hasgarage', 'pool']
Following YAML columns are not in the file uploaded ['datepoststring']


0

In [34]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['unnamed_0', 'id', 'stateid', 'countyid', 'cityid', 'country',
       'datepostedstring', 'is_bankowned', 'is_forauction', 'event', 'time',
       'price', 'pricepersquarefoot', 'city', 'state', 'yearbuilt',
       'streetaddress', 'zipcode', 'longitude', 'latitude', 'hasbadgeocode',
       'description', 'currency', 'livingarea', 'livingareavalue',
       'lotareaunits', 'bathrooms', 'bedrooms', 'buildingarea', 'parking',
       'garagespaces', 'hasgarage', 'levels', 'pool', 'spa',
       'isnewconstruction', 'haspetsallowed', 'hometype', 'county'],
      dtype='object')
columns of YAML are: ['id', 'stateId', 'countyId', 'cityId', 'country', 'datePostString', 'is_bankOwned', 'is_forAuction']


In [35]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['latitude', 'garagespaces', 'livingarea', 'description', 'livingareavalue', 'county', 'event', 'unnamed_0', 'streetaddress', 'datepostedstring', 'hometype', 'isnewconstruction', 'price', 'longitude', 'city', 'time', 'state', 'lotareaunits', 'hasbadgeocode', 'bathrooms', 'levels', 'parking', 'bedrooms', 'currency', 'zipcode', 'yearbuilt', 'pricepersquarefoot', 'spa', 'buildingarea', 'haspetsallowed', 'hasgarage', 'pool']
Following YAML columns are not in the file uploaded ['datepoststring']
validation failed


In [36]:
pd.read_csv("/gdrive/My Drive/RealEstate_California.csv")

,Unnamed: 0,id,stateId,countyId,cityId,country,datePostedString,is_bankOwned,is_forAuction,event,...,parking,garageSpaces,hasGarage,levels,pool,spa,isNewConstruction,hasPetsAllowed,homeType,county
0,0,95717-2087851113,9,77,24895,USA,2021-01-13,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,LOT,Placer County
1,1,94564-18496265,9,189,36958,USA,2021-07-12,0,0,Listed for sale,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
2,2,94564-18484475,9,190,36958,USA,2021-07-08,0,0,Listed for sale,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
3,3,94564-18494835,9,191,36958,USA,2021-07-07,0,0,Listed for sale,...,1,1.0,1,Two Story,0,1,0,0,SINGLE_FAMILY,Contra Costa County
4,4,94564-2069722747,9,192,36958,USA,2021-07-07,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,LOT,Contra Costa County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35384,35384,90036-20610069,9,130313,12447,USA,2021-06-19,0,0,Listing removed,...,0,0.0,0,0,0,0,0,0,SINGLE_FAMILY,Los Angeles County
35385,35385,90036-20610391,9,130314,12447,USA,2021-06-17,0,0,Listing removed,...,1,2.0,1,0,0,0,0,0,SINGLE_FAMILY,Los Angeles County
35386,35386,95062-16111852,9,130368,13715,USA,2021-06-21,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,SINGLE_FAMILY,Santa Cruz County
35387,35387,95062-16111367,9,130369,13715,USA,2021-06-11,0,0,Listed for sale,...,1,0.0,0,0,0,0,0,0,SINGLE_FAMILY,Santa Cruz County


In [39]:
df

,Unnamed: 0,id,stateId,countyId,cityId,country,datePostedString,is_bankOwned,is_forAuction,event,...,parking,garageSpaces,hasGarage,levels,pool,spa,isNewConstruction,hasPetsAllowed,homeType,county
0,0,95717-2087851113,9,77,24895,USA,2021-01-13,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,LOT,Placer County
1,1,94564-18496265,9,189,36958,USA,2021-07-12,0,0,Listed for sale,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
2,2,94564-18484475,9,190,36958,USA,2021-07-08,0,0,Listed for sale,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
3,3,94564-18494835,9,191,36958,USA,2021-07-07,0,0,Listed for sale,...,1,1.0,1,Two Story,0,1,0,0,SINGLE_FAMILY,Contra Costa County
4,4,94564-2069722747,9,192,36958,USA,2021-07-07,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,LOT,Contra Costa County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35384,35384,90036-20610069,9,130313,12447,USA,2021-06-19,0,0,Listing removed,...,0,0.0,0,0,0,0,0,0,SINGLE_FAMILY,Los Angeles County
35385,35385,90036-20610391,9,130314,12447,USA,2021-06-17,0,0,Listing removed,...,1,2.0,1,0,0,0,0,0,SINGLE_FAMILY,Los Angeles County
35386,35386,95062-16111852,9,130368,13715,USA,2021-06-21,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,SINGLE_FAMILY,Santa Cruz County
35387,35387,95062-16111367,9,130369,13715,USA,2021-06-11,0,0,Listed for sale,...,1,0.0,0,0,0,0,0,0,SINGLE_FAMILY,Santa Cruz County


In [55]:
### Creating test file for this demo:
testdata = {
    'id' : ['95717-2087851113', '90036-20610069', '95060-16100392'],
    'stateId' : [9,9,9],
    'countyId' : [77, 130313, 130370],
    'cityId' : [24895, 12447, 13715],
    'country' : ['USA', 'USA', 'USA'],
    'datePostString' : ['2021-01-13', '2021-06-19', '2021-06-08'],
    'is_bankOwned' : [0, 0, 0],
    'is_forAuction' : [0, 0, 0]

}
import pandas as pd
df = pd.DataFrame(testdata, columns=['id', 'stateId', 'countyId', 'cityId', 'country', 'datePostString', 'is_bankOwned', 'is_forAuction'])
df.to_csv("/gdrive/My Drive/RealEstate_California.csv",index=False)

In [56]:
df

,id,stateId,countyId,cityId,country,datePostString,is_bankOwned,is_forAuction
0,95717-2087851113,9,77,24895,USA,2021-01-13,0,0
1,90036-20610069,9,130313,12447,USA,2021-06-19,0,0
2,95060-16100392,9,130370,13715,USA,2021-06-08,0,0


In [57]:
testdata

{'cityId': [24895, 12447, 13715],
 'country': ['USA', 'USA', 'USA'],
 'countyId': [77, 130313, 130370],
 'datePostString': ['2021-01-13', '2021-06-19', '2021-06-08'],
 'id': ['95717-2087851113', '90036-20610069', '95060-16100392'],
 'is_bankOwned': [0, 0, 0],
 'is_forAuction': [0, 0, 0],
 'stateId': [9, 9, 9]}